# Analyse d'Impact COVID-19 : Système Hospitalier PSL

## Objectifs de l'étude

Cette analyse compare les données hospitalières **pré-COVID** (2010-2017) avec les données **pendant COVID** (2019-2021) pour :

1. **Mesurer l'impact** de la pandémie sur les stocks, lits et matériaux
2. **Calculer les marges de changement** pour chaque ressource critique
3. **Développer une stratégie** de préparation aux futures pandémies
4. **Proposer des recommandations** pour l'allocation des ressources

Les résultats permettront d'identifier les vulnérabilités du système et de construire une résilience pour l'avenir.

## 1. Import des bibliothèques et chargement des données

In [1]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
%matplotlib inline

print("✓ Bibliothèques importées avec succès")

✓ Bibliothèques importées avec succès


In [2]:
# Chargement des datasets
print("Chargement des données...")

# Dataset pré-COVID (2010-2016)
df_pre_covid = pd.read_csv('dataset_hebdo_2010-2016.csv')
print(f"✓ Données pré-COVID chargées: {df_pre_covid.shape}")

# Dataset COVID (2019-2021)  
covid_file_path = '/Users/martinvalentine/Downloads/dataset_hebdo_psl_covid_2019_2021.csv'
df_covid = pd.read_csv(covid_file_path)
print(f"✓ Données COVID chargées: {df_covid.shape}")

print("\nStructure des données pré-COVID:")
print("Colonnes:", df_pre_covid.columns.tolist())
df_pre_covid.head()

Chargement des données...
✓ Données pré-COVID chargées: (366, 4)
✓ Données COVID chargées: (157, 4)

Structure des données pré-COVID:
Colonnes: ['Date', 'Passages_Hebdo', 'Lits_Capacite', 'Indice_Tension']


,Date,Passages_Hebdo,Lits_Capacite,Indice_Tension
0,2010-01-04,890,2100,6.1
1,2010-01-11,1560,2100,10.6
2,2010-01-18,1560,2100,10.6
3,2010-01-25,1560,2100,10.6
4,2010-02-01,1570,2100,10.7


In [3]:
print("Structure des données COVID:")
print("Colonnes:", df_covid.columns.tolist())
df_covid.head()

Structure des données COVID:
Colonnes: ['Date', 'Passages_Hebdo', 'Lits_Capacite', 'Indice_Tension']


,Date,Passages_Hebdo,Lits_Capacite,Indice_Tension
0,2019-01-07,2740,2250,17.4
1,2019-01-14,2740,2250,17.4
2,2019-01-21,2740,2250,17.4
3,2019-01-28,2740,2250,17.4
4,2019-02-04,2760,2250,17.5


## 2. Exploration et nettoyage des données

In [4]:
# Preprocessing et harmonisation des datasets
def clean_and_process_data(df, dataset_name):
    """Nettoie et prépare les données pour l'analyse"""
    df_clean = df.copy()
    
    # Conversion des dates
    if 'Date' in df_clean.columns:
        df_clean['Date'] = pd.to_datetime(df_clean['Date'])
    elif 'date' in df_clean.columns:
        df_clean['Date'] = pd.to_datetime(df_clean['date'])
    
    # Extraction des composantes temporelles
    df_clean['year'] = df_clean['Date'].dt.year
    df_clean['month'] = df_clean['Date'].dt.month
    df_clean['week'] = df_clean['Date'].dt.isocalendar().week
    
    print(f"\n{dataset_name}:")
    print(f"  Période: {df_clean['Date'].min()} → {df_clean['Date'].max()}")
    print(f"  Années: {sorted(df_clean['year'].unique())}")
    print(f"  Valeurs manquantes: {df_clean.isnull().sum().sum()}")
    
    return df_clean

# Nettoyage des deux datasets
df_pre_covid_clean = clean_and_process_data(df_pre_covid, "Données pré-COVID")
df_covid_clean = clean_and_process_data(df_covid, "Données COVID")


Données pré-COVID:
  Période: 2010-01-04 00:00:00 → 2017-01-02 00:00:00
  Années: [np.int32(2010), np.int32(2011), np.int32(2012), np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017)]
  Valeurs manquantes: 0

Données COVID:
  Période: 2019-01-07 00:00:00 → 2022-01-03 00:00:00
  Années: [np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022)]
  Valeurs manquantes: 0


## 3. Analyse des données pré-COVID (baseline)

In [5]:
# Établissement des métriques baseline (pré-COVID)
baseline_metrics = {}

# Identification des colonnes communes pour l'analyse
common_cols = [col for col in df_pre_covid_clean.columns if col in df_covid_clean.columns]
print("Colonnes communes pour l'analyse:", common_cols)

# Calcul des métriques baseline sur toute la période pré-COVID
baseline_period = df_pre_covid_clean

baseline_metrics = {
    'Passages_Hebdo': {
        'mean': baseline_period['Passages_Hebdo'].mean(),
        'std': baseline_period['Passages_Hebdo'].std(),
        'min': baseline_period['Passages_Hebdo'].min(),
        'max': baseline_period['Passages_Hebdo'].max(),
        'median': baseline_period['Passages_Hebdo'].median()
    },
    'Lits_Capacite': {
        'mean': baseline_period['Lits_Capacite'].mean(),
        'std': baseline_period['Lits_Capacite'].std(),
        'min': baseline_period['Lits_Capacite'].min(),
        'max': baseline_period['Lits_Capacite'].max(),
        'median': baseline_period['Lits_Capacite'].median()
    },
    'Indice_Tension': {
        'mean': baseline_period['Indice_Tension'].mean(),
        'std': baseline_period['Indice_Tension'].std(),
        'min': baseline_period['Indice_Tension'].min(),
        'max': baseline_period['Indice_Tension'].max(),
        'median': baseline_period['Indice_Tension'].median()
    }
}

print("MÉTRIQUES BASELINE (2010-2016)")
print("=" * 50)
for metric, values in baseline_metrics.items():
    print(f"\n{metric}:")
    for stat, value in values.items():
        print(f"  {stat}: {value:.2f}")

Colonnes communes pour l'analyse: ['Date', 'Passages_Hebdo', 'Lits_Capacite', 'Indice_Tension', 'year', 'month', 'week']
MÉTRIQUES BASELINE (2010-2016)

Passages_Hebdo:
  mean: 1858.09
  std: 470.87
  min: 890.00
  max: 3230.00
  median: 1840.00

Lits_Capacite:
  mean: 2165.70
  std: 47.86
  min: 2100.00
  max: 2230.00
  median: 2160.00

Indice_Tension:
  mean: 12.22
  std: 2.90
  min: 6.10
  max: 20.70
  median: 12.20


## 4. Analyse de la période COVID (2020-2021)

In [7]:
# Analyse de l'impact COVID (2020-2021)
covid_period = df_covid_clean[df_covid_clean['year'].isin([2020, 2021])]
pre_covid_2019 = df_covid_clean[df_covid_clean['year'] == 2019]

print(f"Données COVID période: {len(covid_period)} observations")
print(f"Données pré-COVID 2019: {len(pre_covid_2019)} observations")

# Calcul des métriques COVID
covid_metrics = {}
pre_2019_metrics = {}

for metric in ['Passages_Hebdo', 'Lits_Capacite', 'Indice_Tension']:
    if metric in covid_period.columns:
        covid_metrics[metric] = {
            'mean': covid_period[metric].mean(),
            'std': covid_period[metric].std(),
            'min': covid_period[metric].min(),
            'max': covid_period[metric].max(),
            'median': covid_period[metric].median()
        }
        
        pre_2019_metrics[metric] = {
            'mean': pre_2019_2019[metric].mean() if len(pre_covid_2019) > 0 else 0,
            'std': pre_covid_2019[metric].std() if len(pre_covid_2019) > 0 else 0,
            'min': pre_covid_2019[metric].min() if len(pre_covid_2019) > 0 else 0,
            'max': pre_covid_2019[metric].max() if len(pre_covid_2019) > 0 else 0,
            'median': pre_covid_2019[metric].median() if len(pre_covid_2019) > 0 else 0
        }

print("\nMÉTRIQUES PÉRIODE COVID (2020-2021)")
print("=" * 50)
for metric, values in covid_metrics.items():
    print(f"\n{metric}:")
    for stat, value in values.items():
        print(f"  {stat}: {value:.2f}")

Données COVID période: 104 observations
Données pré-COVID 2019: 52 observations


NameError: name 'pre_2019_2019' is not defined

## 5. Analyse comparative : Pré-COVID vs COVID

In [ ]:
# Calcul des changements (% et absolus)
impact_analysis = {}

print("ANALYSE COMPARATIVE D'IMPACT COVID")
print("=" * 60)

for metric in baseline_metrics.keys():
    if metric in covid_metrics:
        baseline_mean = baseline_metrics[metric]['mean']
        covid_mean = covid_metrics[metric]['mean']
        
        # Changement absolu et relatif
        absolute_change = covid_mean - baseline_mean
        relative_change = (absolute_change / baseline_mean) * 100 if baseline_mean != 0 else 0
        
        # Volatilité (changement de variabilité)
        baseline_std = baseline_metrics[metric]['std']
        covid_std = covid_metrics[metric]['std']
        volatility_change = ((covid_std - baseline_std) / baseline_std) * 100 if baseline_std != 0 else 0
        
        impact_analysis[metric] = {
            'baseline_mean': baseline_mean,
            'covid_mean': covid_mean,
            'absolute_change': absolute_change,
            'relative_change': relative_change,
            'volatility_change': volatility_change,
            'baseline_std': baseline_std,
            'covid_std': covid_std
        }
        
        print(f"\n📊 {metric}:")
        print(f"   Baseline (2010-2016): {baseline_mean:.1f} ± {baseline_std:.1f}")
        print(f"   COVID (2020-2021):    {covid_mean:.1f} ± {covid_std:.1f}")
        print(f"   Changement absolu:     {absolute_change:+.1f}")
        print(f"   Changement relatif:    {relative_change:+.1f}%")
        print(f"   Changement volatilité: {volatility_change:+.1f}%")
        
        # Interprétation
        if abs(relative_change) > 20:
            impact_level = "🔴 IMPACT MAJEUR"
        elif abs(relative_change) > 10:
            impact_level = "🟡 IMPACT MODÉRÉ"
        else:
            impact_level = "🟢 IMPACT FAIBLE"
        print(f"   Niveau d'impact:       {impact_level}")

## 6. Calcul des marges de changement pour stocks et matériel

In [ ]:
# Calcul des marges de sécurité nécessaires
print("CALCUL DES MARGES DE SÉCURITÉ POUR FUTURES PANDÉMIES")
print("=" * 65)

safety_margins = {}

for metric, analysis in impact_analysis.items():
    baseline_mean = analysis['baseline_mean']
    covid_max = covid_metrics[metric]['max']
    covid_min = covid_metrics[metric]['min']
    
    # Calcul des marges de sécurité
    max_surge_capacity = ((covid_max - baseline_mean) / baseline_mean) * 100
    min_reduction_tolerance = ((baseline_mean - covid_min) / baseline_mean) * 100
    
    # Marge de sécurité recommandée (avec buffer de 20%)
    recommended_buffer_up = max_surge_capacity * 1.2
    recommended_buffer_down = min_reduction_tolerance * 1.2
    
    safety_margins[metric] = {
        'max_surge_observed': max_surge_capacity,
        'max_reduction_observed': min_reduction_tolerance,
        'recommended_buffer_up': recommended_buffer_up,
        'recommended_buffer_down': recommended_buffer_down,
        'baseline_value': baseline_mean,
        'recommended_max_capacity': baseline_mean * (1 + recommended_buffer_up/100),
        'recommended_min_reserve': baseline_mean * (1 - recommended_buffer_down/100)
    }
    
    print(f"\n🏥 {metric}:")
    print(f"   Valeur baseline:              {baseline_mean:.1f}")
    print(f"   Pic maximal observé COVID:    {covid_max:.1f} ({max_surge_capacity:+.1f}%)")
    print(f"   Minimum observé COVID:        {covid_min:.1f} ({-min_reduction_tolerance:+.1f}%)")
    print(f"   🎯 RECOMMANDATIONS PANDÉMIE:")
    print(f"      Capacité maximale requise: {safety_margins[metric]['recommended_max_capacity']:.1f}")
    print(f"      Réserve minimale requise:  {safety_margins[metric]['recommended_min_reserve']:.1f}")
    print(f"      Marge haute (+20% buffer): +{recommended_buffer_up:.1f}%")
    print(f"      Marge basse (+20% buffer): -{recommended_buffer_down:.1f}%")

## 7. Tests de significativité statistique

In [ ]:
# Tests statistiques pour valider la significativité des changements
print("TESTS DE SIGNIFICATIVITÉ STATISTIQUE")
print("=" * 50)

statistical_results = {}

for metric in baseline_metrics.keys():
    if metric in covid_metrics:
        # Données pour les tests
        baseline_data = df_pre_covid_clean[metric].dropna()
        covid_data = covid_period[metric].dropna()
        
        # Test t de Student pour comparer les moyennes
        t_stat, t_pvalue = stats.ttest_ind(baseline_data, covid_data)
        
        # Test de Levene pour comparer les variances
        levene_stat, levene_pvalue = stats.levene(baseline_data, covid_data)
        
        # Test de Kolmogorov-Smirnov pour comparer les distributions
        ks_stat, ks_pvalue = stats.ks_2samp(baseline_data, covid_data)
        
        statistical_results[metric] = {
            't_statistic': t_stat,
            't_pvalue': t_pvalue,
            'levene_statistic': levene_stat,
            'levene_pvalue': levene_pvalue,
            'ks_statistic': ks_stat,
            'ks_pvalue': ks_pvalue
        }
        
        print(f"\n📈 {metric}:")
        print(f"   Test t (moyennes):     t={t_stat:.3f}, p={t_pvalue:.4f}")
        if t_pvalue < 0.05:
            print(f"   ✅ Différence significative des moyennes (p < 0.05)")
        else:
            print(f"   ❌ Différence non significative des moyennes (p ≥ 0.05)")
            
        print(f"   Test Levene (variances): stat={levene_stat:.3f}, p={levene_pvalue:.4f}")
        if levene_pvalue < 0.05:
            print(f"   ✅ Différence significative des variances (p < 0.05)")
        else:
            print(f"   ❌ Différence non significative des variances (p ≥ 0.05)")
            
        print(f"   Test KS (distributions): stat={ks_stat:.3f}, p={ks_pvalue:.4f}")
        if ks_pvalue < 0.05:
            print(f"   ✅ Distributions significativement différentes (p < 0.05)")
        else:
            print(f"   ❌ Distributions non significativement différentes (p ≥ 0.05)")

## 8. Visualisations des tendances clés

In [ ]:
# Visualisations complètes
fig, axes = plt.subplots(3, 2, figsize=(20, 18))
fig.suptitle('ANALYSE COMPARATIVE PRÉ-COVID vs COVID : IMPACT SUR LE SYSTÈME HOSPITALIER', fontsize=16, fontweight='bold')

metrics = ['Passages_Hebdo', 'Lits_Capacite', 'Indice_Tension']

for i, metric in enumerate(metrics):
    # Graphique 1: Évolution temporelle
    ax1 = axes[i, 0]
    
    # Données pré-COVID
    pre_covid_monthly = df_pre_covid_clean.groupby([df_pre_covid_clean['Date'].dt.to_period('M')])[metric].mean()
    covid_monthly = df_covid_clean.groupby([df_covid_clean['Date'].dt.to_period('M')])[metric].mean()
    
    ax1.plot(pre_covid_monthly.index.astype(str), pre_covid_monthly.values, 
             label='Pré-COVID (2010-2016)', linewidth=2, color='blue', alpha=0.7)
    ax1.plot(covid_monthly.index.astype(str), covid_monthly.values, 
             label='COVID (2019-2021)', linewidth=2, color='red', alpha=0.7)
    
    ax1.axhline(y=baseline_metrics[metric]['mean'], color='blue', linestyle='--', alpha=0.5, label='Moyenne pré-COVID')
    ax1.axhline(y=covid_metrics[metric]['mean'], color='red', linestyle='--', alpha=0.5, label='Moyenne COVID')
    
    ax1.set_title(f'{metric} - Évolution temporelle')
    ax1.set_ylabel(metric)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Rotation des labels de l'axe x pour une meilleure lisibilité
    for tick in ax1.get_xticklabels():
        tick.set_rotation(45)
    
    # Graphique 2: Comparaison des distributions
    ax2 = axes[i, 1]
    
    baseline_data = df_pre_covid_clean[metric].dropna()
    covid_data = covid_period[metric].dropna()
    
    ax2.hist(baseline_data, bins=30, alpha=0.7, label='Pré-COVID', color='blue', density=True)
    ax2.hist(covid_data, bins=30, alpha=0.7, label='COVID', color='red', density=True)
    
    ax2.axvline(baseline_data.mean(), color='blue', linestyle='--', linewidth=2, label=f'Moyenne pré-COVID: {baseline_data.mean():.1f}')
    ax2.axvline(covid_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Moyenne COVID: {covid_data.mean():.1f}')
    
    ax2.set_title(f'{metric} - Distribution des valeurs')
    ax2.set_xlabel(metric)
    ax2.set_ylabel('Densité')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Stratégie de préparation aux pandémies futures

In [ ]:
# Développement de la stratégie pandémique basée sur les données
print("🎯 STRATÉGIE DE PRÉPARATION AUX FUTURES PANDÉMIES")
print("=" * 70)

pandemic_strategy = {
    'immediate_actions': [],
    'medium_term_planning': [],
    'long_term_investments': [],
    'monitoring_indicators': []
}

# Analyse des besoins par métrique
for metric, margins in safety_margins.items():
    baseline = margins['baseline_value']
    max_capacity_needed = margins['recommended_max_capacity']
    min_reserve_needed = margins['recommended_min_reserve']
    
    print(f"\n🏥 STRATÉGIE POUR {metric}:")
    print(f"   Situation actuelle baseline: {baseline:.1f}")
    print(f"   Capacité maximale requise:   {max_capacity_needed:.1f}")
    print(f"   Réserve minimale requise:    {min_reserve_needed:.1f}")
    
    # Actions immédiates
    immediate_surge = max_capacity_needed - baseline
    immediate_buffer = (immediate_surge / baseline) * 100
    
    print(f"\n   📋 ACTIONS IMMÉDIATES:")
    print(f"      • Créer une capacité de surge: +{immediate_surge:.1f} unités ({immediate_buffer:.1f}%)")
    print(f"      • Établir des stocks d'urgence: {min_reserve_needed:.1f} unités minimum")
    print(f"      • Mettre en place des protocoles d'activation rapide")
    
    # Planning moyen terme
    print(f"\n   📊 PLANNING MOYEN TERME (6-18 mois):")
    print(f"      • Développer des partenariats pour capacité supplémentaire")
    print(f"      • Former le personnel aux protocoles pandémie")
    print(f"      • Installer des systèmes de monitoring en temps réel")
    
    # Investissements long terme
    print(f"\n   🏗️ INVESTISSEMENTS LONG TERME (2-5 ans):")
    print(f"      • Infrastructure modulaire extensible")
    print(f"      • Systèmes d'information prédictifs")
    print(f"      • Réseaux de collaboration inter-hospitaliers")

print(f"\n" + "=" * 70)
print("🚨 INDICATEURS D'ALERTE PRÉCOCE À SURVEILLER")
print("=" * 70)

# Définition des seuils d'alerte basés sur l'analyse
alert_thresholds = {}
for metric, margins in safety_margins.items():
    baseline = margins['baseline_value']
    
    # Seuils d'alerte (percentiles basés sur l'analyse COVID)
    yellow_threshold = baseline * 1.10  # +10% = alerte jaune
    orange_threshold = baseline * 1.25  # +25% = alerte orange
    red_threshold = baseline * 1.40     # +40% = alerte rouge
    
    alert_thresholds[metric] = {
        'baseline': baseline,
        'yellow': yellow_threshold,
        'orange': orange_threshold,
        'red': red_threshold
    }
    
    print(f"\n🔍 {metric}:")
    print(f"   ✅ Normal:      < {yellow_threshold:.1f}")
    print(f"   🟡 Vigilance:   {yellow_threshold:.1f} - {orange_threshold:.1f}")
    print(f"   🟠 Attention:   {orange_threshold:.1f} - {red_threshold:.1f}")
    print(f"   🔴 Critique:    > {red_threshold:.1f}")

## 10. Recommandations d'allocation des ressources

In [ ]:
# Recommandations finales d'allocation des ressources
print("💰 RECOMMANDATIONS D'ALLOCATION DES RESSOURCES")
print("=" * 60)

# Calcul du budget d'investissement basé sur l'analyse
total_investment = {}
priority_matrix = {}

for metric, margins in safety_margins.items():
    baseline = margins['baseline_value']
    surge_needed = margins['recommended_max_capacity'] - baseline
    
    # Estimation des coûts (exemple - à adapter selon les coûts réels)
    if metric == 'Lits_Capacite':
        cost_per_unit = 10000  # Coût d'un lit supplémentaire équipé
        investment_type = "Infrastructure hospitalière"
    elif metric == 'Passages_Hebdo':
        cost_per_unit = 500    # Coût par capacité de passage supplémentaire
        investment_type = "Capacité d'accueil"
    else:
        cost_per_unit = 1000   # Coût générique par unité
        investment_type = "Ressources générales"
    
    total_cost = surge_needed * cost_per_unit
    
    # Priorité basée sur l'impact observé
    if metric in impact_analysis:
        impact_magnitude = abs(impact_analysis[metric]['relative_change'])
        if impact_magnitude > 30:
            priority = "🔴 TRÈS HAUTE"
            priority_score = 4
        elif impact_magnitude > 20:
            priority = "🟠 HAUTE"
            priority_score = 3
        elif impact_magnitude > 10:
            priority = "🟡 MOYENNE"
            priority_score = 2
        else:
            priority = "🟢 BASSE"
            priority_score = 1
    else:
        priority = "🟡 MOYENNE"
        priority_score = 2
    
    total_investment[metric] = {
        'surge_capacity_needed': surge_needed,
        'cost_per_unit': cost_per_unit,
        'total_cost': total_cost,
        'investment_type': investment_type,
        'priority': priority,
        'priority_score': priority_score
    }
    
    print(f"\n💡 {metric} ({investment_type}):")
    print(f"   Capacité de surge nécessaire: {surge_needed:.1f} unités")
    print(f"   Coût estimé par unité:        {cost_per_unit:,} €")
    print(f"   Investissement total:         {total_cost:,.0f} €")
    print(f"   Priorité:                     {priority}")

# Calcul du budget total et priorisation
total_budget = sum([inv['total_cost'] for inv in total_investment.values()])
print(f"\n" + "=" * 60)
print(f"💰 BUDGET TOTAL ESTIMÉ: {total_budget:,.0f} €")
print(f"=" * 60)

# Priorisation des investissements
print(f"\n📊 PRIORISATION DES INVESTISSEMENTS:")
sorted_investments = sorted(total_investment.items(), 
                          key=lambda x: x[1]['priority_score'], 
                          reverse=True)

for i, (metric, inv_data) in enumerate(sorted_investments, 1):
    percentage = (inv_data['total_cost'] / total_budget) * 100
    print(f"{i}. {inv_data['priority']} {metric}")
    print(f"   Budget: {inv_data['total_cost']:,.0f} € ({percentage:.1f}% du total)")

print(f"\n" + "=" * 60)
print("📋 PLAN D'IMPLÉMENTATION RECOMMANDÉ")
print("=" * 60)

print("""
🎯 PHASE 1 (0-6 mois) - Actions immédiates:
   • Établir les protocoles d'urgence
   • Créer les stocks de réserve stratégique
   • Former les équipes aux procédures pandémie
   • Budget estimé: 20% du total

🎯 PHASE 2 (6-18 mois) - Développement des capacités:
   • Développer la capacité de surge identifiée
   • Installer les systèmes de monitoring
   • Établir les partenariats inter-hospitaliers
   • Budget estimé: 50% du total

🎯 PHASE 3 (18-36 mois) - Infrastructure durable:
   • Investir dans l'infrastructure modulaire
   • Développer les systèmes prédictifs
   • Créer les réseaux de collaboration régionaux
   • Budget estimé: 30% du total

✅ MÉTRIQUES DE SUCCÈS:
   • Temps de réponse < 24h pour activation de la capacité de surge
   • Maintien des stocks de réserve au niveau requis
   • Réduction de 50% du temps de détection des signaux d'alerte
   • Capacité de surge testée trimestriellement
""")